In [2]:
from langchain.document_loaders import UnstructuredMarkdownLoader
loader = UnstructuredMarkdownLoader("C:\\Users\\qianwens\\tsg.md")
data = loader.load()


Install torch requires python 3.8.6 using below nightly build
pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu118

In [7]:

import os
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

OPENAI_API_KEY= ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = "https://qianwenai.openai.azure.com"


text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings)






Created a chunk of size 4246, which is longer than the specified 1000
Created a chunk of size 5242, which is longer than the specified 1000
Created a chunk of size 1316, which is longer than the specified 1000
Created a chunk of size 4766, which is longer than the specified 1000
Created a chunk of size 15208, which is longer than the specified 1000
Created a chunk of size 2423, which is longer than the specified 1000
Created a chunk of size 4357, which is longer than the specified 1000
Created a chunk of size 18245, which is longer than the specified 1000
Created a chunk of size 1687, which is longer than the specified 1000
Created a chunk of size 35616, which is longer than the specified 1000
Created a chunk of size 4810, which is longer than the specified 1000
Created a chunk of size 1790, which is longer than the specified 1000
Created a chunk of size 1934, which is longer than the specified 1000
Created a chunk of size 1862, which is longer than the specified 1000
Created a chunk o

InvalidRequestError: The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.

In [ ]:

llm = ChatOpenAI(
model_name="gpt-3.5-turbo",
temperature=0,
openai_api_key=OPENAI_API_KEY,
max_tokens=512,
verbose=True
)